In [3]:
from music21 import converter,instrument,note,chord,stream
import glob
import pickle
import numpy as np
from keras.utils import np_utils

# Read a Midi File

In [4]:
midi=converter.parse("midi/alb_esp1.mid")

In [5]:
midi

<music21.stream.Score 0x23d5ffb1340>

In [6]:
midi.show('midi')

In [7]:
#midi.show('text') #we use midi and not mp3 since music21 can process midi files only

In [8]:
elements_to_parse=midi.flat.notes

In [9]:
len(elements_to_parse)

606

In [10]:
notes_demo=[]
for e in elements_to_parse:
    #if the element is a note, then store its a pitch
    if isinstance(e, note.Note):
        notes_demo.append(str(e.pitch))
    
    
    #if the element is a chord,split each note of chord and join them with +
    elif isinstance(e, chord.Chord):
        notes_demo.append("+".join(str(n) for n in e.normalOrder))

In [11]:
str(elements_to_parse[3].pitch)

'E4'

In [12]:
isinstance(elements_to_parse[7], note.Note)

False

In [13]:
len(notes_demo)

606

# Preprocessing List

In [ ]:
notes=[]
for file in glob.glob("midi/*.mid"):
    midi=converter.parse(file)
    
    print("parsing %s"%file)
    
    elements_to_parse=midi.flat.notes
    for e in elements_to_parse:
        if isinstance(e, note.Note):
            notes.append(str(e.pitch))
        elif isinstance(e, chord.Chord):
            notes.append("+".join(str(n) for n in e.normalOrder))

In [ ]:
with open("notes","wb") as filepath:
    pickle.dump(notes,filepath)

In [14]:
with open("notes","rb") as file:
    notes=pickle.load(file)

In [15]:
n_vocab=len(set(notes)) #no.of classes

In [16]:
print(n_vocab)

386


In [17]:
len(notes)

86481

# SEQUENTIAL DATA  FOR LSTM

MARKOV CHAIN GENERATION

In [18]:
sequence_length=100

In [19]:
pitchnames=sorted(set(notes))

In [20]:
#Mapping between elements and integer values
element_to_int=dict((ele,num)for num,ele in enumerate(pitchnames))
int_to_element=dict((num,ele)for num,ele in enumerate(pitchnames))

In [21]:
network_input=[]
network_output=[]

In [22]:
for i in range(len(notes)-sequence_length):
    seq_in=notes[i:i+sequence_length]
    seq_out=notes[i+sequence_length]
    
    network_input.append([element_to_int[ch] for ch in seq_in])
    network_output.append(element_to_int[seq_out])
    

In [23]:
n_patterns=len(network_input)

In [24]:
n_patterns

86381

In [25]:
network_input=np.reshape(network_input,(n_patterns,sequence_length,1))
print (network_input.shape)

(86381, 100, 1)


In [26]:
normalised_network_input=network_input/float(n_vocab)

In [27]:
#convert into one hot vectors
network_output=np_utils.to_categorical(network_output)
print(network_output.shape)

(86381, 386)


MODEL CREATION

In [28]:
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [29]:
model=Sequential()
model.add(LSTM(units=512,
              input_shape=(normalised_network_input.shape[1],normalised_network_input.shape[2]),
              return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab,activation='softmax'))

In [30]:
model.compile(loss="categorical_crossentropy",optimizer='adam')

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 512)          1052672   
                                                                 
 dropout (Dropout)           (None, 100, 512)          0         
                                                                 
 lstm_1 (LSTM)               (None, 100, 512)          2099200   
                                                                 
 dropout_1 (Dropout)         (None, 100, 512)          0         
                                                                 
 lstm_2 (LSTM)               (None, 512)               2099200   
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0

In [ ]:
checkpoint=ModelCheckpoint("model.hdf5",monitor='loss',verbose=0,save_best_only=True,mode="min")
model.hist=model.fit(normalised_network_input,network_output,steps_per_epoch=100,epochs=100,batch_size=64,callbacks=[checkpoint])

Epoch 1/100
100/100 [==============================] - 56511s 571s/step - loss: 5.0621
Epoch 2/100
100/100 [==============================] - 644s 6s/step - loss: 4.8922
Epoch 3/100
100/100 [==============================] - 624s 6s/step - loss: 4.8498
Epoch 4/100
100/100 [==============================] - 402s 4s/step - loss: 4.8547
Epoch 5/100
100/100 [==============================] - 489s 5s/step - loss: 4.7947
Epoch 6/100
100/100 [==============================] - 615s 6s/step - loss: 4.7542
Epoch 7/100
100/100 [==============================] - 623s 6s/step - loss: 4.8988
Epoch 8/100
100/100 [==============================] - 592s 6s/step - loss: 4.8216
Epoch 9/100
100/100 [==============================] - 561s 6s/step - loss: 4.7917
Epoch 10/100
100/100 [==============================] - 518s 5s/step - loss: 4.7661
Epoch 11/100
100/100 [==============================] - 540s 5s/step - loss: 4.7456
Epoch 12/100
  2/100 [..............................] - ETA: 8:44 - loss: 4.8295

In [32]:
from keras.models import load_model

In [34]:
model=load_model("model.hdf5")

PREDICTIONS

In [35]:
network_input=[]

for i in range(len(notes)-sequence_length):
    seq_in=notes[i:i+sequence_length]
    network_input.append([element_to_int[ch] for ch in seq_in])
    

In [42]:
start=np.random.randint(len(network_input)-1)

In [43]:
int_to_ele=dict((num,ele) for num,ele in enumerate(pitchnames))

In [44]:
pattern=network_input[start]
prediction_output=[]
#generate 200 elements
for not_index in range(200):
    prediction_input=np.reshape(pattern,(1,len(pattern),1))
    prediction_input=prediction_input/float(n_vocab)
    prediction=model.predict(prediction_input,verbose=0)
    idx=np.argmax(prediction)
    result=int_to_ele[idx]
    prediction_output.append(result)
    pattern.append(idx)
    pattern=pattern[1:]

# CREATE MIDI FILE

In [45]:
offset=0 #time
output_notes=[]

for pattern in prediction_output:
    #if the pattern is a chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord=pattern.split('+')
        temp_notes=[]
        for current_note in notes_in_chord:
            new_note=note.Note(int(current_note)) #create note object for each note in chord
            new_note.storedInstrument=instrument.Piano()
            temp_notes.append(new_note)
            
        new_chord=chord.Chord(temp_notes)
        new_chord.offset=offset
        output_notes.append(new_chord)
        
    else:
        #if the pattern is a note
        new_note=note.Note(pattern) #create note object for each note in chord
        new_note.offset=offset
        new_note.storedInstrument=instrument.Piano()
        output_notes.append(new_note)
        
    offset+=0.5 #all elements are played after same time
    

In [46]:
#create a stream object
midi_stream=stream.Stream(output_notes)
midi_stream.write('midi',fp='test_output.mid')

'test_output.mid'

In [47]:
midi_stream.show('midi')

DISADVANTAGES:
    1. doesnt know how it should start or end
    2. offset is taken as constant